In [1]:
# Proportional Odds Logistic Regression for Ordered Category Outcomes

In [2]:
pip install --upgrade statsmodels


Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
import numpy as np
from math import log
from scipy.stats import chi2


import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.miscmodels.ordinal_model import OrderedModel

In [29]:
# read the soccer dataset
soccer_df = pd.read_csv("soccer.csv")

# preview the first few rows
soccer_df.head()


,discipline,n_yellow_25,n_red_25,position,result,country,level
0,NaN,4,1,S,D,England,1
1,NaN,2,2,D,W,England,2
2,NaN,2,1,M,D,England,1
3,NaN,2,1,M,L,Germany,1
4,NaN,2,0,S,W,Germany,1


In [30]:
# treat discipline as an ordered categorical variable that reflects the increasing severity of referee action:
# None < Yellow < Red

# fill NaN with "None" before converting to ordered categorical
soccer_df["discipline"] = soccer_df["discipline"].fillna("None")

# ensure discipline is now ordinal
# define the ordered levels
discipline_order = pd.CategoricalDtype(categories=["None", "Yellow", "Red"], ordered=True)

# convert the discipline column to an ordered categorical variable
soccer_df["discipline"] = soccer_df["discipline"].astype(discipline_order)

# verify
soccer_df["discipline"].value_counts()

discipline
None      1562
Yellow     455
Red        274
Name: count, dtype: int64

In [31]:
# verify the conversion
soccer_df["discipline"].dtype

CategoricalDtype(categories=['None', 'Yellow', 'Red'], ordered=True, categories_dtype=object)

In [32]:
# convert position, country, result, and level to categorical fields

# define categorical columns
cat_cols = ["position", "country", "result", "level"]

# convert each to categorical dtype
soccer_df[cat_cols] = soccer_df[cat_cols].apply(lambda x: x.astype("category"))

# verify structure
soccer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   discipline   2291 non-null   category
 1   n_yellow_25  2291 non-null   int64   
 2   n_red_25     2291 non-null   int64   
 3   position     2291 non-null   category
 4   result       2291 non-null   category
 5   country      2291 non-null   category
 6   level        2291 non-null   category
dtypes: category(5), int64(2)
memory usage: 47.7 KB


In [33]:
# view
soccer_df.head()

,discipline,n_yellow_25,n_red_25,position,result,country,level
0,None,4,1,S,D,England,1
1,None,2,2,D,W,England,2
2,None,2,1,M,D,England,1
3,None,2,1,M,L,Germany,1
4,None,2,0,S,W,Germany,1


In [34]:
# rebuild model_data *after* restoring None
model_data = soccer_df[["discipline", "n_yellow_25", "n_red_25", "position", "country", "level", "result"]].copy()

# check
model_data["discipline"].value_counts()


discipline
None      1562
Yellow     455
Red        274
Name: count, dtype: int64

In [35]:

# Encode categorical predictors using dummy variables
X = pd.get_dummies(
    model_data[["n_yellow_25", "n_red_25", "position", "country", "level", "result"]],
    drop_first=True,
    dtype=float
)

# Dependent variable (ordered)
y = model_data["discipline"].astype(
    pd.CategoricalDtype(categories=["None", "Yellow", "Red"], ordered=True)
)

# Fit proportional odds model (logistic link)
model = OrderedModel(y, X, distr="logit")
res = model.fit(method="bfgs", disp=False)

# Show results
print(res.summary())


                             OrderedModel Results                             
Dep. Variable:             discipline   Log-Likelihood:                -1722.3
Model:                   OrderedModel   AIC:                             3465.
Method:            Maximum Likelihood   BIC:                             3522.
Date:                Tue, 11 Nov 2025                                         
Time:                        15:42:00                                         
No. Observations:                2291                                         
Df Residuals:                    2281                                         
Df Model:                           8                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
n_yellow_25         0.3224      0.033      9.745      0.000       0.258       0.387
n_red_25            0.3832      0.041

In [36]:
# Extract coefficient results
coefs = res.params
std_err = res.bse
z_vals = res.tvalues
p_vals = res.pvalues

# Compute odds ratios
odds_ratios = np.exp(coefs)

# Combine all into a dataframe
summary_table = pd.DataFrame({
    "Coefficient": coefs,
    "Std. Error": std_err,
    "z value": z_vals,
    "p value": p_vals,
    "Odds Ratio": odds_ratios
})

# Display rounded version
summary_table.round(4)


,Coefficient,Std. Error,z value,p value,Odds Ratio
n_yellow_25,0.3224,0.0331,9.7453,0.0000,1.3804
n_red_25,0.3832,0.0405,9.4613,0.0000,1.4670
position_M,0.1969,0.1165,1.6901,0.0910,1.2176
position_S,-0.6853,0.1501,-4.5654,0.0000,0.5039
country_Germany,0.1330,0.0936,1.4210,0.1553,1.1423
level_2,0.0910,0.0935,0.9726,0.3308,1.0952
result_L,0.4830,0.1120,4.3146,0.0000,1.6210
result_W,-0.7395,0.1213,-6.0967,0.0000,0.4774
None/Yellow,2.5085,0.1918,13.0769,0.0000,12.2863
Yellow/Red,0.3487,0.0441,7.9048,0.0000,1.4172


In [37]:
# Fit a reduced (parsimonious) model
# drop predictors that are not statistically significant

# define reduced feature set
X_reduced = pd.get_dummies(
    model_data[["n_yellow_25", "n_red_25", "position", "result"]],
    drop_first=True,
    dtype=float
)

# response variable (ordered)
y = model_data["discipline"].astype(
    pd.CategoricalDtype(categories=["None", "Yellow", "Red"], ordered=True)
)

# fit reduced proportional odds model
model_reduced = OrderedModel(y, X_reduced, distr="logit")
res_reduced = model_reduced.fit(method="bfgs", disp=False)

print(res_reduced.summary())


                             OrderedModel Results                             
Dep. Variable:             discipline   Log-Likelihood:                -1723.7
Model:                   OrderedModel   AIC:                             3463.
Method:            Maximum Likelihood   BIC:                             3509.
Date:                Tue, 11 Nov 2025                                         
Time:                        15:54:08                                         
No. Observations:                2291                                         
Df Residuals:                    2283                                         
Df Model:                           6                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
n_yellow_25     0.3222      0.033      9.735      0.000       0.257       0.387
n_red_25        0.3813      0.040      9.425     

In [38]:
# Compare fit with the full model
# AIC values (lower = better fit, accounting for model complexity)
print("Full Model AIC:", res.aic)
print("Reduced Model AIC:", res_reduced.aic)


Full Model AIC: 3464.5339375392336
Reduced Model AIC: 3463.485368098396


In [40]:
# # get fitted probabilities for each observation
predicted_probs = res_reduced.model.predict(res_reduced.params)

# convert to DataFrame for clarity
predicted_probs = pd.DataFrame(predicted_probs, columns=["None", "Yellow", "Red"])

# view first few rows
predicted_probs.head(30)


,None,Yellow,Red
0,0.803135,0.140706,0.056159
1,0.848940,0.109656,0.041405
2,0.763607,0.166499,0.069894
3,0.665679,0.225657,0.108663
4,0.959534,0.030333,0.010133
5,0.769112,0.162966,0.067921
6,0.707055,0.201564,0.091382
7,0.877179,0.089950,0.032871
8,0.441644,0.323532,0.234825
9,0.451320,0.320818,0.227863


In [47]:
# goodness-of-fit tests
# Note: there are numerous types

# Create a dummy constant column with all zeros (not ones!)
null_X = np.zeros((len(y), 1))  # constant 0 column
null_model = OrderedModel(y, null_X, distr="logit")
null_res = null_model.fit(method="bfgs", disp=False)

# Log-likelihoods
ll_model = res_reduced.llf
ll_null = null_res.llf
n = res_reduced.model.nobs

# McFadden, Cox-Snell, Nagelkerke
r2_mcfadden = 1 - (ll_model / ll_null)
r2_coxsnell = 1 - np.exp((2 / n) * (ll_null - ll_model))
r2_nagelkerke = r2_coxsnell / (1 - np.exp((2 / n) * ll_null))

print(f"McFadden:   {r2_mcfadden:.4f}")
print(f"Cox–Snell:  {r2_coxsnell:.4f}")
print(f"Nagelkerke: {r2_nagelkerke:.4f}")
print(f"AIC:        {res_reduced.aic:.4f}")


print("""
Model diagnostics summary:
- McFadden R² ≈ 0.10  → moderate fit (typical for social data)
- Cox–Snell R² ≈ 0.15 → consistent with moderate improvement
- Nagelkerke R² ≈ 0.19 → normalized version; confirms same pattern
- AIC ≈ 3463.5 → slightly improved over full model (better parsimony)
""")


McFadden:   0.1002
Cox–Snell:  0.1542
Nagelkerke: 0.1899
AIC:        3463.4854

Model diagnostics summary:
- McFadden R² ≈ 0.10  → moderate fit (typical for social data)
- Cox–Snell R² ≈ 0.15 → consistent with moderate improvement
- Nagelkerke R² ≈ 0.19 → normalized version; confirms same pattern
- AIC ≈ 3463.5 → slightly improved over full model (better parsimony)



C:\Users\rasha\anaconda3\Lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [50]:
# check the proportional odds assumption
# i.e. whether the effect (slope) of each predictor is roughly constant across all thresholds of the ordinal outcome
# Assumptions check method 1: run stratified binomial models on the data 

# create binary variables
soccer_df["yellow_plus"] = np.where(soccer_df["discipline"] == "None", 0, 1)
soccer_df["red"] = np.where(soccer_df["discipline"] == "Red", 1, 0)



# model: "Any card" vs "No card"
yellowplus_model = smf.logit(
    "yellow_plus ~ n_yellow_25 + n_red_25 + position + result + country + level",
    data=soccer_df
).fit(disp=False)

# model: "Red card" vs "Yellow/None"
red_model = smf.logit(
    "red ~ n_yellow_25 + n_red_25 + position + result + country + level",
    data=soccer_df
).fit(disp=False)


In [51]:
# check the proportional odds assumption
# i.e. whether the effect (slope) of each predictor is roughly constant across all thresholds of the ordinal outcome
# Assumptions check method 1: run stratified binomial models on the data 

# Compare coefficients (check similarity)
# collect estimates
coef_yellowplus = yellowplus_model.params
coef_red = red_model.params

# combine into a dataframe
coef_comparison = pd.DataFrame({
    "yellowplus": coef_yellowplus,
    "red": coef_red,
    "difference": coef_red - coef_yellowplus
})

# display nicely
coef_comparison.round(4)


,yellowplus,red,difference
Intercept,-2.6365,-3.8987,-1.2622
position[T.M],0.2611,0.0639,-0.1972
position[T.S],-0.7212,-0.4423,0.2789
result[T.L],0.4616,0.6430,0.1813
result[T.W],-0.7782,-0.5854,0.1929
country[T.Germany],0.1314,0.1080,-0.0234
level[T.2],0.0806,0.1242,0.0436
n_yellow_25,0.3459,0.3247,-0.0212
n_red_25,0.4145,0.3421,-0.0724


In [59]:
# check the proportional odds assumption
# i.e. whether the effect (slope) of each predictor is roughly constant across all thresholds of the ordinal outcome

# Assumptions check method 2: Brant-Wald Test

# check the proportional odds assumption
# i.e. whether the effect (slope) of each predictor is roughly constant across thresholds

# outcome
y = soccer_df["discipline"]

# predictors (include categorical variables)
X = soccer_df[["n_yellow_25", "n_red_25", "position", "result", "country", "level"]]

# convert categorical columns to dummy variables (drop_first to avoid multicollinearity)
X_encoded = pd.get_dummies(X, drop_first=True)

# convert boolean columns to int (critical fix)
X_encoded = X_encoded.astype(float)

# verify that all predictors are numeric
print(X_encoded.dtypes)

# fit proportional odds (ordinal) model
model = OrderedModel(y, X_encoded, distr="logit")
res = model.fit(method="bfgs", disp=False)




n_yellow_25        float64
n_red_25           float64
position_M         float64
position_S         float64
result_L           float64
result_W           float64
country_Germany    float64
level_2            float64
dtype: object


In [61]:
# check the proportional odds assumption
# i.e. whether the effect (slope) of each predictor is roughly constant across all thresholds of the ordinal outcome

# Assumptions check method 2: Brant-Wald Test



LR statistic: 0.000
Degrees of freedom: 1
p-value: 1.0000
